# Derivative of traits_i with respect to abundance_i and abundance_k

Here I will be testing my solutions for
$\frac{ \partial \mathbf{\hat{V}}_i }{ \partial N_i }$
and
$\frac{ \partial \mathbf{\hat{V}}_i }{ \partial N_k }$
(see below)
by calculating the Jacobian using the `theano` package
and comparing those results to my solution.


## Importing packages and setting options

In [2]:
%env OMP_NUM_THREADS=4
%env THEANO_FLAGS='openmp=True'
import sympy
import theano
theano.config.cxx = ""
import theano.tensor as T
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
pd.options.display.max_columns = 10

env: OMP_NUM_THREADS=4
env: THEANO_FLAGS='openmp=True'


## Equations

__Notes:__

- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- Multiplication between matrices is always matrix multiplication, not
  element-wise
  

Below are the equations for
(1) traits for species $i$ at time $t+1$ ($\mathbf{V}_{i,t+1}$),
(2) the partial derivative of species $i$ traits at time $t+1$ with respect 
to species $i$ abundance at time $t$, and
(3) the partial derivative of species $i$ traits at time $t+1$ with respect 
to species $k$ abundance at time $t$.

\begin{align}
\mathbf{V}_{i,t+1} &= \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \alpha_0 \left( 
                N_{i,t} +
                N_{k,t} \; \textrm{e}^{ - \mathbf{V}_{k,t}^\textrm{T} \mathbf{D} \mathbf{V}_{k,t} } +
            \sum_{j \ne i, j \ne k}^{n}{ N_{j,t} \; \textrm{e}^{ - \mathbf{V}_{j,t}^\textrm{T} \mathbf{D} \mathbf{V}_{j,t} } }
            \right) ~
            \textrm{e}^{-\mathbf{V}_{i,t}^\textrm{T} \mathbf{V}_{i,t}} ~ \mathbf{V}_{i,t}^\textrm{T}
        - f ~ \mathbf{V}_{i,t}^\textrm{T} \mathbf{C}
    \right] \\
    \frac{ \partial \mathbf{V}_{i,t+1} }{ \partial N_{i,t} } &= 2 ~ \sigma_i^2 ~ \alpha_0 ~ 
          \mathbf{V}_{i,t} ~
          \textrm{e}^{ - \mathbf{V}_{i,t}^{\textrm{T}} \mathbf{V}_{i,t} } \\
    \frac{ \partial \mathbf{V}_{i,t+1} }{ \partial N_{k,t} } &= 2 ~ \sigma_i^2 ~ \alpha_0 ~ 
          \mathbf{V}_{i,t} ~
          \textrm{e}^{ -\mathbf{V}_{k,t}^\textrm{T} \mathbf{D} \mathbf{V}_{k,t} - 
                        \mathbf{V}_{i,t}^{\textrm{T}} \mathbf{V}_{i,t} } \\
\end{align}


## Read CSV of simulated datasets

In [3]:
sims = pd.read_csv("simulated_data.csv")
sims.head()

,V1,V2,V3,V4,V5,...,f,a0,eta,r0,d
0,4.945110,2.869199,6.747126,6.142522,5.629532,...,0.068890,0.112113,-0.331150,1.422746,-0.091228
1,0.718846,1.220364,0.815571,0.868633,0.838021,...,0.309021,0.057579,0.094811,1.237047,0.003429
2,3.369285,1.912974,3.131174,0.046303,1.416252,...,0.118318,0.401410,-0.036977,1.746024,0.012160
3,0.373669,0.283873,0.237735,0.053632,0.062281,...,0.497286,0.499730,0.117188,0.669199,0.081612
4,3.562637,1.635016,5.724176,4.953962,1.060083,...,0.042638,0.307171,-0.467453,0.952351,0.051834


---------


# V_i / N_i


## Functions to compare methods

In [4]:
def automatic(i, V, N, D, C, f, a0, s2):
    """Automatic differentiation using theano pkg"""
    Vi = V[:,i]
    Nj_vec = np.array([N[j] * np.exp(-V[:,j].T @ D @ V[:,j]) for j in range(N.size) if j != i])
    Ni = T.dscalar('Ni')
    Vhat = Vi + 2 * s2 * (
        ( a0 * (Ni + T.sum(Nj_vec)) * T.exp(-1 * T.dot(Vi.T, Vi)) * Vi.T) - 
        ( f * T.dot(Vi.T, C) )
    )
    J, updates = theano.scan(lambda i, Vhat, Ni : T.grad(Vhat[i], Ni), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Ni])
    num_fun = theano.function([Ni], J, updates=updates)
    out_array = num_fun(N[i])
    return out_array

In [19]:
def symbolic(i, V, N, D, C, f, a0, s2):
    """Symbolic differentiation using math"""
    q = V.shape[0]
    I = np.identity(q)
    Vi = V[:,i]
    Vi = Vi.reshape((q, 1))
    dVhat = 2 * s2 * a0 * Vi @ np.exp(-1 * Vi.T @ Vi)
    # Using flatten just bc automatic returns a 1D vector:
    return dVhat.flatten()

In [20]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, d)
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    
    # Create output array:
    diffs = np.empty((n, 4))
    diffs[:,0] = sim_i
    
    # Fill output array:
    for i in range(n):
        auto = automatic(i, V, N, D, C, f, a0, s2)
        sym =   symbolic(i, V, N, D, C, f, a0, s2)
        if abs:
            diff = auto - sym
        else:
            diff = (auto - sym) / sym
        diff = diff.flatten()
        diffs[i, 1] = i
        diffs[i, 2] = diff.min()
        diffs[i, 3] = diff.max()
    
    return diffs

### Example of using `compare_methods`:

In [21]:
diffs = compare_methods(0)
# Worst case examples:
print(diffs[:,2].min())
print(diffs[:,3].max())

-1.830604910320897e-16
0.0


## Comparing methods

This takes ~2 minutes.

In [22]:
n_per_rep = 4
diffs = np.empty((int(n_per_rep * 100), 4))

In [23]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep, abs = True)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

100%|██████████| 100/100 [01:45<00:00,  1.06s/it]


## The results
They appear to be extremely similar, enough so that I feel comfortable with my symbolic version.

In [24]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-4.336808689942018e-19
8.673617379884035e-19


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [25]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
# Output array
results = np.zeros((100, n * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, d)
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)

    # Fill output array:
    for i in range(0, n):
        sym =   symbolic(i, V, N, D, C, f, a0, s2)
        results[sim_i, (i*q):((i+1)*q)] = sym.flatten()

# Make sure first and last aren't zeros:
results[[0, 99], :]

array([[1.19849875e-36, 6.95380145e-37, 1.63523596e-36, 3.38465312e-34,
        3.10198537e-34, 1.01024677e-34, 9.73617680e-21, 8.21932008e-21,
        1.12997543e-20, 2.10101626e-10, 3.03198443e-10, 2.70721931e-10],
       [5.41886815e-04, 1.82026473e-04, 1.32173719e-04, 1.15430834e-10,
        9.89930051e-11, 2.70905096e-11, 8.94585202e-04, 4.01578882e-04,
        2.77397908e-04, 1.57622658e-09, 3.06988476e-08, 2.12230414e-08]])

In [26]:
np.savetxt('results/dVi_dNi.csv', results, delimiter=',')

---------


# V_i / N_k


## Functions to compare methods

In [27]:
def automatic(i, k, V, N, D, C, f, a0, s2):
    """Automatic differentiation using theano pkg"""
    Vi = V[:,i]
    Vk = V[:,k]
    Nj = np.sum(np.array([N[j] * np.exp(-V[:,j].T @ D @ V[:,j]) for j in range(N.size) if j != i and j != k]))
    Ni = N[i]
    Nk = T.dscalar('Nk')
    Vhat = Vi + 2 * s2 * (
        ( a0 * (Ni + Nk * T.exp(-1 * T.dot(T.dot(Vk.T, D), Vk)) + Nj) * 
         T.exp(-1 * T.dot(Vi.T, Vi)) * Vi.T) - 
        ( f * T.dot(Vi.T, C) )
    )
    J, updates = theano.scan(lambda i, Vhat, Nk : T.grad(Vhat[i], Nk), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Nk])
    num_fun = theano.function([Nk], J, updates=updates)
    out_array = num_fun(N[k])
    return out_array

In [28]:
def symbolic(i, k, V, N, D, C, f, a0, s2):
    """Symbolic differentiation using math"""
    q = V.shape[0]
    Vi = V[:,i]
    Vi = Vi.reshape((q, 1))
    Vk = V[:,k]
    Vk = Vk.reshape((q, 1))
    dVhat = 2 * s2 * a0 * Vi @ np.exp(-1 * Vk.T @ D @ Vk) * np.exp(- Vi.T @ Vi)
    # Using flatten just bc automatic returns a 1D vector:
    return dVhat.flatten()

In [29]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, d)
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    
    # Create output array:
    diffs = np.empty((math.factorial(n) // math.factorial(n-2), 4))
    j = 0
    for i in range(0, n):
        for k in [x for x in range(0, n) if x != i]:
            num = automatic(i, k, V, N, D, C, f, a0, s2)
            sym = symbolic(i, k, V, N, D, C, f, a0, s2)
            num = num.flatten()
            sym = sym.flatten()
            if abs:
                diff = num - sym
            else:
                diff = (num - sym) / sym
                if np.any(sym == 0):
                    for l in [x for x in range(0, diff.size) if sym[x] == 0]:
                        diff[l] = num[l];
            diffs[j, 0] = i
            diffs[j, 1] = k
            diffs[j, 2] = diff.min()
            diffs[j, 3] = diff.max()
            j += 1
    return diffs

### Example of using `compare_methods`:

In [30]:
diffs = compare_methods(0)
# Worst case examples:
print(diffs[:,2].min())
print(diffs[:,3].max())

-2.5261016395749876e-16
0.0


## Comparing methods

This takes ~2 minutes.

In [31]:
n_per_rep = math.factorial(4) // math.factorial(4-2)
diffs = np.empty((int(n_per_rep * 100), 4))

In [32]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep, abs = True)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

100%|██████████| 100/100 [07:26<00:00,  4.46s/it]


## The results
They appear to be extremely similar, enough so that I feel comfortable with my symbolic version.

In [33]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-1.4210854715202004e-14
2.842170943040401e-14


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [34]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
n_perms = math.factorial(n) // math.factorial(n-2)
# Output array
results = np.zeros((100, n_perms * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, d)
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    
    # Fill output array:
    j = 0
    for i in range(0, n):
        for k in [x for x in range(0, n) if x != i]:
            sym = symbolic(i, k, V, N, D, C, f, a0, s2)
            results[sim_i, (j*q):((j+1)*q)] = sym.flatten()
            j += 1


# Make sure first and last aren't zeros:
results[[0, 99], :]

array([[9.16907604e-34, 5.31998338e-34, 1.25103200e-33, 5.17750033e-35,
        3.00403395e-35, 7.06419989e-35, 5.55760956e-36, 3.22457687e-36,
        7.58282228e-36, 4.24815957e-31, 3.89337648e-31, 1.26798503e-31,
        1.46216612e-32, 1.34005399e-32, 4.36425404e-33, 1.56951191e-33,
        1.43843484e-33, 4.68465831e-34, 1.22201098e-17, 1.03162664e-17,
        1.41825935e-17, 7.44863068e-18, 6.28816433e-18, 8.64484059e-18,
        4.51480398e-20, 3.81141590e-20, 5.23985716e-20, 2.63703606e-07,
        3.80551662e-07, 3.39789609e-07, 1.60737571e-07, 2.31960992e-07,
        2.07114940e-07, 9.07636522e-09, 1.30981367e-08, 1.16951552e-08],
       [9.44076844e-05, 3.17127070e-05, 2.30273453e-05, 4.28445414e-04,
        1.43920105e-04, 1.04503786e-04, 1.57630608e-04, 5.29500678e-05,
        3.84482944e-05, 8.59221157e-11, 7.36864509e-11, 2.01650966e-11,
        9.12659432e-11, 7.82692952e-11, 2.14192416e-11, 3.35779207e-11,
        2.87962858e-11, 7.88041598e-12, 6.65893599e-04, 2.98919

In [35]:
np.savetxt('results/dVi_dNk.csv', results, delimiter=',')